In [1]:
from pathlib import Path
import pandas as pd
from pandas import DataFrame, MultiIndex, merge, read_csv, concat
import seaborn as sns

from reservoirs_synthetic_bph.utils.data import get_dataframe, remove_warmup_df
from reservoirs_synthetic_bph.utils.global_config import (
    N_WARMUPS,
    SERIES,
    TSTEPS,
    DATA_DIR,
    TEST_FILE,
    METRIC_CSV_FILE,
)
from reservoirs_synthetic_bph.utils.post_processing import (
    METRIC,
    DSET,
    TGT,
    VAL,
    SIMS,
    SERIES as INDIV,
    TSTEPS,
    PRED,
)

MDL = "model"
METD = "method"
SPEC = "spec."
NOISE = "w/ noise"
NONOISE = "w/o noise"

Data folder: /home/francois/Documents/SISTM/subSISTM/data/synthetic_bph_1


In [2]:
def generate_display(
    df: pd.DataFrame, metric: str, effect: str, column_order: list = None
):
    # we use pivot to make sur that there's a single value for each case
    # pivot_table use aggregating and will return a value instead of raising an error

    idxs = [MDL, SPEC]
    cols = [DSET, TGT]

    if METD in df.columns:
        cols = [METD] + cols

    df = df[df[METRIC] == metric]
    assert len(df) > 0

    df = df[df[DSET].str.endswith(effect)]
    assert len(df) > 0

    df = (
        df.pivot(
            index=idxs,
            columns=cols,
            values=[VAL],
        )
        .droplevel(level=0, axis=1)
        .sort_index(level=0, axis=1, ascending=False)
    )

    if column_order:
        df = df.reindex(columns=pd.MultiIndex.from_product(column_order))

    stlr = df.style
    stlr.format("{:#.2g}")
    # use df.to_html() to find the CSS labels
    stlr.set_table_styles(
        [
            {
                "selector": "table, th, td",
                "props": "border: 1px solid; text-align: center",
            },
        ]
    )
    # stlr.background_gradient(axis=0)
    return stlr


def base_convert_name(name, conversion_list):
    result = [nc for nc in conversion_list if name in nc[1]]
    assert len(result) == 1, name
    return result[0][0]

In [3]:
test_pred = read_csv(
    DATA_DIR + "/01_test.csv", sep=";", decimal=",", index_col=[INDIV, TSTEPS]
)[["y_fixed", "y_mixed"]]
test_pred

y_fixed    y_mixed
individus temps                      
1         0     -10.270811 -14.147090
          1      -9.577035 -13.233794
          2      -9.241133 -12.791824
          3      -9.046099 -12.532934
          4      -8.925675 -12.371850
...                    ...        ...
500       21      3.614660  14.255805
          22      3.974612  14.365780
          23      4.330982  14.480369
          24      4.683909  14.599297
          25      5.033576  14.722291

[13000 rows x 2 columns]

# Arthur's results

In [4]:
# put a comma at the end of a single value tuple


def arthur_convert_name(name):
    names_conversions = [
        (
            ("Linear model", "x1+…+x8"),
            ("lin_simple_fixed",),
        ),
        (
            ("Linear model", "x2*x5+x4*x7+x6*x8"),
            ("fixed",),
        ),
        (
            ("Mixed model", "x2*x5+x4*x7+x6*x8"),
            ("mixed",),
        ),
        (
            ("Mixed model", "x1+…+x8"),
            ("lin_mixed", "lin_fixed"),
        ),
        (
            ("Mixed model", "t^1+…+t^4"),
            ("naif_fixed", "naif_mixed"),
        ),
    ]
    return base_convert_name(name, names_conversions)

## Metrics

In [5]:
arthur_metrics = pd.read_csv(
    DATA_DIR + "/Résultats/Performances_moyennes.csv", index_col=0
).drop(
    columns=["mse_test_fixed_obs.1"]
)  # I've checked: it a duplicate
# there is also "Résultats simulation.csv" with MSE/MAE for each simulation

In [6]:
def split_names(c: str) -> dict[str, str]:
    splt = c.split("_")
    model_ = "_".join(splt[2:-1])
    model, spec = arthur_convert_name(model_)
    return {
        METRIC: splt[0],
        DSET: f"{splt[1]}_{splt[-2]}",
        MDL: model,
        SPEC: spec,
        TGT: splt[-1],
    }


arthur_metrics = pd.DataFrame(
    [
        {**split_names(col), VAL: arthur_metrics[col].iloc[0]}
        for col in arthur_metrics.columns
    ]
)

arthur_metrics["target"] = arthur_metrics["target"].replace(
    {"truth": NONOISE, "obs": NOISE, "obs.1": NOISE}
)

In [7]:
arthur_metrics

,metric,dataset,model,spec.,target,value
0,mae,train_mixed,Mixed model,x2*x5+x4*x7+x6*x8,w/o noise,0.211769
1,mse,train_mixed,Mixed model,x2*x5+x4*x7+x6*x8,w/o noise,0.084689
2,mae,test_mixed,Mixed model,x2*x5+x4*x7+x6*x8,w/o noise,0.209960
3,mse,test_mixed,Mixed model,x2*x5+x4*x7+x6*x8,w/o noise,0.084497
4,mae,train_mixed,Mixed model,t^1+…+t^4,w/o noise,1.480595
5,mse,train_mixed,Mixed model,t^1+…+t^4,w/o noise,8.234363
6,mae,test_mixed,Mixed model,t^1+…+t^4,w/o noise,1.434460
7,mse,test_mixed,Mixed model,t^1+…+t^4,w/o noise,8.285175
8,mae,train_mixed,Mixed model,x2*x5+x4*x7+x6*x8,w/ noise,0.760418
9,mse,train_mixed,Mixed model,x2*x5+x4*x7+x6*x8,w/ noise,0.915951


In [8]:
# generate_display(arthur_metrics, metric="mse", effect="fixed")

In [9]:
# generate_display(arthur_metrics, metric="mse", effect="mixed")

## Predictions

Arthur's prediction are the ones made on the test dataset with the models trained on the different simulations.

In [10]:
arthur_preds = pd.read_csv(
    DATA_DIR + "/Résultats/Predictions.csv.gz", index_col=[INDIV, TSTEPS]
)
arthur_preds = arthur_preds.drop(columns=["Unnamed: 0"])
arthur_preds = arthur_preds.melt(ignore_index=False)

arthur_preds = arthur_preds.rename(columns={"variable": SIMS})
arthur_preds[SIMS] = arthur_preds[SIMS].str.replace(r"^pred_", r"", regex=True)
arthur_preds[MDL] = (
    arthur_preds[SIMS]
    .str.replace(r"_\d+$", r"", regex=True)
    .apply(arthur_convert_name)
    .str.join(r"\n")
)


def get_effect(x):
    for effect in ["mixed", "fixed"]:
        if effect in x:
            return effect
    raise UserWarning()


arthur_preds[DSET] = "test-" + arthur_preds[SIMS].apply(get_effect)
arthur_preds[SIMS] = arthur_preds[SIMS].str.replace(
    r"^.*_(\d+)", r"simulation\1.csv", regex=True
)

arthur_preds = arthur_preds.set_index(SIMS, append=True)
arthur_preds = arthur_preds.set_index(MDL, append=True)
arthur_preds = arthur_preds.set_index(DSET, append=True)
arthur_preds = arthur_preds.reorder_levels([DSET, MDL, SIMS, INDIV, TSTEPS])

arthur_preds = arthur_preds.rename(columns={"value": PRED})
arthur_preds

pred
dataset    model                          simulation        individus temps           
test-mixed Mixed model\nx2*x5+x4*x7+x6*x8 simulation1.csv   1         0     -14.825990
                                                                      1     -13.545691
                                                                      2     -12.922801
                                                                      3     -12.592934
                                                                      4     -12.406496
...                                                                                ...
test-fixed Mixed model\nt^1+…+t^4         simulation100.csv 500       21      2.911111
                                                                      22      3.158828
                                                                      23      3.406546
                                                                      24      3.654263
                                                                      25      3.901981

[9100000 rows x 1 columns]

# Results with a forecast method for the mixed model

The prediction of a mixed model normally uses the data at all timesteps.  

Here to predict the value for the timesteps $t0$ we only use the data of the previous steps.

Here's the corresponding R code:

```R
# initialization with the marginal predictions
pred <- as.vector(predictY(model, newdata = data, marg = TRUE)$pred)
for (t in temps[-1:-1]) {
# using only the previous time steps to predict the random effect
prev_data <- data[data[TSTEP] < t, ]
ui <- predictRE(model, prev_data)
# (some checks…)
# combining the marginal and the random effects prediction == subject specific
reffects <- ui$intercept + rowSums(data[data[TSTEP] == t, X_LABELS] * ui[, X_LABELS])
pred[data[TSTEP] == t] <- pred[data[TSTEP] == t] + reffects
}
```

In [11]:
# put a comma at the end of a single value tuple


def new_convert_name(name):
    names_conversions = [
        (
            ("Mixed model", "x2*x5+x4*x7+x6*x8"),
            ("mixed_oracle"),
        ),
        (
            ("Mixed model", "x1+…+x8"),
            ("mixed_linear", "fixed_linear"),
        ),
        (
            ("Mixed model", "t^1+…+t^4"),
            ("mixed_time-polynom", "fixed_time-polynom"),
        ),
    ]
    return base_convert_name(name, names_conversions)

## Metrics

In [12]:
new_metrics = pd.DataFrame()
for pth in Path("mixed_model_forecast").rglob("*csv"):
    model = str(pth).split("/")[1]
    tmp = pd.read_csv(pth, index_col=0)
    tmp[MDL] = model
    new_metrics = pd.concat([new_metrics, tmp])

In [13]:
def new_restructure_row(row):
    try:
        if row[TGT].endswith("_obs"):
            tgt = NOISE
        else:
            tgt = NONOISE
    except AttributeError:
        tgt = row[TGT]

    dset = row[DSET] + "_" + row[MDL].split("_")[0]
    mdl = new_convert_name(row[MDL])
    met = (
        row[METRIC]
        .replace("mean_absolute_error", "mae")
        .replace("mean_squared_error", "mse")
    )

    row[DSET] = dset
    row[TGT] = tgt
    row[METRIC] = met
    row[MDL], row[SPEC] = mdl
    return row


new_metrics = new_metrics.apply(new_restructure_row, axis=1)
new_metrics

,dataset,target,metric,value,model,spec.
0,train_mixed,w/o noise,mae,0.316157,Mixed model,x2*x5+x4*x7+x6*x8
1,train_mixed,w/o noise,mse,0.175762,Mixed model,x2*x5+x4*x7+x6*x8
2,train_mixed,w/ noise,mae,0.863870,Mixed model,x2*x5+x4*x7+x6*x8
3,train_mixed,w/ noise,mse,1.174989,Mixed model,x2*x5+x4*x7+x6*x8
4,test_mixed,w/o noise,mae,0.313691,Mixed model,x2*x5+x4*x7+x6*x8
5,test_mixed,w/o noise,mse,0.173205,Mixed model,x2*x5+x4*x7+x6*x8
6,test_mixed,w/ noise,mae,0.856459,Mixed model,x2*x5+x4*x7+x6*x8
7,test_mixed,w/ noise,mse,1.157790,Mixed model,x2*x5+x4*x7+x6*x8
0,test_mixed,w/o noise,quad-bias,0.172845,Mixed model,x2*x5+x4*x7+x6*x8
1,test_mixed,w/ noise,quad-bias,1.157430,Mixed model,x2*x5+x4*x7+x6*x8


In [14]:
# generate_display(new_metrics, metric="mse", effect="fixed")

In [15]:
# generate_display(new_metrics, metric="mse", effect="mixed")

## Predictions

In [16]:
new_preds = pd.DataFrame()
for pth in Path("mixed_model_forecast").rglob("*gz"):
    model = str(pth).split("/")[1]
    tmp = pd.read_csv(pth, index_col=[SIMS, INDIV, TSTEPS])
    tmp = tmp[tmp[DSET] == "test"]
    tmp[DSET] += f'-{model.split("_")[0]}'
    tmp[MDL] = r"\n".join(new_convert_name(model))
    tmp = tmp.set_index([MDL, DSET], append=True)
    new_preds = pd.concat([new_preds, tmp])

new_preds = new_preds.reorder_levels([DSET, MDL, SIMS, INDIV, TSTEPS])
new_preds

pred
dataset    model                          simulation       individus temps           
test-mixed Mixed model\nx2*x5+x4*x7+x6*x8 simulation1.csv  1         0     -11.187728
                                                                     1     -15.257036
                                                                     2     -13.879837
                                                                     3     -13.277482
                                                                     4     -12.881533
...                                                                               ...
           Mixed model\nx1+…+x8           simulation99.csv 500       21     14.951400
                                                                     22     14.727796
                                                                     23     14.658540
                                                                     24     14.847263
                                                                     25     14.464328

[6500000 rows x 1 columns]

# Reservoirs results

In [17]:
# put a comma at the end of a single value tuple


def reservoir_convert_name(name):
    names_conversions = [
        (
            ("Reservoir model", "x2*x5+x4*x7+x6*x8"),
            ("SF_ME", "SF_FE"),
        ),
        (
            ("Reservoir model", "x2*x5+x4*x7+x6*x8+y(t-1)"),
            ("SF_ME_y", "SF_FE_y"),
        ),
        (
            ("Reservoir model", "x1+…+x8"),
            ("AF_ME", "AF_FE"),
        ),
        (
            ("Reservoir model", "x1+…+x8+y(t-1)"),
            ("AF_ME_y", "AF_FE_y"),
        ),
    ]
    return base_convert_name(name, names_conversions)

## Metrics

In [18]:
reservoir_metrics = pd.DataFrame()
for pth in Path("reservoirs_synthetic_bph").rglob("metrics.csv"):
    model = str(pth).split("/")[1]
    tmp = pd.read_csv(pth, index_col=0)
    tmp[MDL] = model
    reservoir_metrics = pd.concat([reservoir_metrics, tmp])
reservoir_metrics

,dataset,target,metric,value,model
0,train,y_mixed,mean_absolute_error,0.554750,SF_ME_y
1,train,y_mixed,mean_squared_error,0.515345,SF_ME_y
2,train,y_mixed_obs,mean_absolute_error,0.974460,SF_ME_y
3,train,y_mixed_obs,mean_squared_error,1.506047,SF_ME_y
4,test,y_mixed,mean_absolute_error,0.574440,SF_ME_y
...,...,...,...,...,...
6,test,y_mixed_obs,mean_absolute_error,10.799250,AF_ME
7,test,y_mixed_obs,mean_squared_error,275.186608,AF_ME
0,test,y_mixed,quad-bias,267.439341,AF_ME
1,test,y_mixed_obs,quad-bias,268.376239,AF_ME


In [19]:
def reservoir_restructure_row(row):
    try:
        if row[TGT].endswith("_obs"):
            tgt = NOISE
        else:
            tgt = NONOISE
    except AttributeError:
        tgt = row[TGT]

    tmp = row[MDL].split("_")[1]
    if tmp == "ME":
        dset = row[DSET] + "_mixed"
    elif tmp == "FE":
        dset = row[DSET] + "_fixed"
    else:
        raise UserWarning(tmp)

    mdl = reservoir_convert_name(row[MDL])
    met = (
        row[METRIC]
        .replace("mean_absolute_error", "mae")
        .replace("mean_squared_error", "mse")
    )

    row[DSET] = dset
    row[TGT] = tgt
    row[METRIC] = met
    row[MDL], row[SPEC] = mdl
    return row


reservoir_metrics = reservoir_metrics.apply(reservoir_restructure_row, axis=1)

In [20]:
generate_display(reservoir_metrics, metric="mse", effect="fixed")

In [21]:
generate_display(reservoir_metrics, metric="mse", effect="mixed")

## Predictions

In [22]:
reservoir_preds = pd.DataFrame()
for pth in Path("reservoirs_synthetic_bph/").rglob("*gz"):
    model = str(pth).split("/")[1]
    tmp = pd.read_csv(pth, index_col=[SIMS, INDIV, TSTEPS])
    tmp = tmp[tmp[DSET] == "test"]
    tmp = tmp.drop(columns=DSET)
    tmp[MDL] = r"\n".join(reservoir_convert_name(model))
    tmp = tmp.set_index([MDL], append=True)
    #
    assert len(tmp.columns) == 1
    pred_col_name = tmp.columns[0]
    tmp = tmp.rename(columns={pred_col_name: PRED})
    #
    reservoir_preds = pd.concat([reservoir_preds, tmp])

reservoir_preds = reservoir_preds.reorder_levels([MDL, SIMS, INDIV, TSTEPS])
reservoir_preds

pred
model                                     simulation       individus temps           
Reservoir model\nx2*x5+x4*x7+x6*x8+y(t-1) simulation71.csv 1         1      -4.404009
                                                                     2      -5.800417
                                                                     3      -6.883582
                                                                     4      -8.624600
                                                                     5     -11.129086
...                                                                               ...
Reservoir model\nx1+…+x8                  simulation50.csv 500       21      0.211994
                                                                     22      0.373328
                                                                     23      0.534336
                                                                     24      0.694264
                                                                     25      0.852549

[7700000 rows x 1 columns]

# Comparison between "fit" and "forecast" mixed model predictions.

## Metrics

In [23]:
copy_1 = arthur_metrics.copy()
copy_1[METD] = "fit"
copy_2 = new_metrics.copy()
copy_2[METD] = "forecast"


comp_df = pd.concat([copy_1, copy_2])
comp_df = comp_df.dropna(subset=[DSET])

In [32]:
generate_display(
    comp_df,
    metric="mse",
    effect="fixed",
    column_order=[
        ("fit", "forecast"),
        ("train_fixed", "test_fixed"),
        ("w/o noise", "w/ noise"),
    ],
)

In [33]:
generate_display(
    comp_df,
    metric="mse",
    effect="mixed",
    column_order=[
        ("fit", "forecast"),
        ("train_mixed", "test_mixed"),
        ("w/o noise", "w/ noise"),
    ],
)

## Predictions

In [ ]:
PROUT

# Updated results

In [ ]:
df_updated = pd.concat(
    [
        arthur_metrics[arthur_metrics[MDL] == "Linear model"],
        new_metrics,
        reservoir_metrics,
    ]
)

In [ ]:
generate_display(df_updated, metric="mse", effect="fixed")

In [ ]:
generate_display(df_updated, metric="mse", effect="mixed")